In [110]:
import math
import pymysql
import pandas as pd
import pprint


import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

# import wikipedia
import spacy
from textblob import TextBlob
from IPython.display import Image
from collections import defaultdict

from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.metrics import confusion_matrix, classification_report
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.decomposition import LatentDirichletAllocation
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.decomposition import TruncatedSVD
from sklearn.linear_model import LogisticRegression
from sklearn.pipeline import Pipeline

from scipy import sparse
from sklearn.metrics import pairwise_distances
from sklearn.metrics.pairwise import cosine_similarity

In [111]:
restaurants_df = pd.read_csv('https://raw.githubusercontent.com/Ajinth/Recommender-System/master/Research/Sample_Restaurants.csv', index_col='Users')

In [112]:
restaurants_df.fillna(value=0, inplace=True)

In [113]:
restaurants_df.head()

,Little_Donkey,Caf__Nero,India_Pavilion,Desi_Dhaba,Thai_Sensation,Dumpling_House,Caf__Luna
Users,,,,,,,
Ajinth,4.0,0.0,5.0,0.0,3.0,0.0,2.0
Tess,3.0,3.0,3.0,0.0,4.0,0.0,0.0
Jessica,4.0,3.0,0.0,0.0,2.0,0.0,1.0
Mark,0.0,2.0,0.0,2.0,0.0,2.0,4.0
Joe,5.0,0.0,5.0,0.0,8.0,0.0,8.0


In [114]:
restaurants_df.loc[restaurants_df.index=='Ajinth']

,Little_Donkey,Caf__Nero,India_Pavilion,Desi_Dhaba,Thai_Sensation,Dumpling_House,Caf__Luna
Users,,,,,,,
Ajinth,4.0,0.0,5.0,0.0,3.0,0.0,2.0


In [115]:
restaurants_df[(restaurants_df > 0).any(axis=1)]

,Little_Donkey,Caf__Nero,India_Pavilion,Desi_Dhaba,Thai_Sensation,Dumpling_House,Caf__Luna
Users,,,,,,,
Ajinth,4.0,0.0,5.0,0.0,3.0,0.0,2.0
Tess,3.0,3.0,3.0,0.0,4.0,0.0,0.0
Jessica,4.0,3.0,0.0,0.0,2.0,0.0,1.0
Mark,0.0,2.0,0.0,2.0,0.0,2.0,4.0
Joe,5.0,0.0,5.0,0.0,8.0,0.0,8.0
Chris,3.0,5.0,0.0,0.0,0.0,4.0,0.0


# Collaborative Item to Item Filtering

In [116]:
'''List of All Restaurants from the Dataframe'''
all_restaurants = list(restaurants_df.columns)
all_restaurants

['Little_Donkey',
 'Caf__Nero',
 'India_Pavilion',
 'Desi_Dhaba',
 'Thai_Sensation',
 'Dumpling_House ',
 'Caf__Luna']

In [117]:
user = 'Tess'

In [118]:
'''Function to return the list of restaurants rated by the user'''
def user_rated_list(user):
    user_rated=[]
    for key, a_res in enumerate(list(restaurants_df.loc[user, :] ==0)):
        if a_res == False:
            user_rated.append((restaurants_df.loc[user, :] ==0).index[key])
        else: 
            continue
    return user_rated   

In [119]:

user_rated = user_rated_list(user)
user_rated

['Little_Donkey', 'Caf__Nero', 'India_Pavilion', 'Thai_Sensation']

In [120]:
'''Function to return the list of restaurants yet to be rated by the user'''
def user_not_rated_list(user):
    user_not_rated=[]
    for key, a_res in enumerate(list(restaurants_df.loc[user, :] ==0)):
        if a_res == True:
            user_not_rated.append((restaurants_df.loc[user, :] ==0).index[key])
        else: 
            continue
    return (user_not_rated) 

In [121]:
user_not_rated = user_not_rated_list(user)
user_not_rated

['Desi_Dhaba', 'Dumpling_House ', 'Caf__Luna']

In [122]:
'''Custom Function to calculate the Co-Similarity'''
def cosim(v1, v2):
    dot_product = sum(n1 * n2 for n1, n2 in zip(v1, v2) )
    magnitude1 = math.sqrt(sum(n ** 2 for n in v1))
    magnitude2 = math.sqrt(sum(n ** 2 for n in v2))
    return dot_product / (magnitude1 * magnitude2)

In [123]:
'''This function calcultes the co-sine similarity between the restaurants rsted by the user and all the restaurants in the list'''
def cooccurence_matrix_creator():
    '''Calling the user rated function'''
    user_rated = user_rated_list(user)
    ur_rest_dict = {}
    for ur in user_rated: 
        ur_cosim=[]
        for ar in list(restaurants_df.columns): 
            similarity = cosim(restaurants_df[ur],\
                                            restaurants_df[ar])
            ur_cosim.append(similarity)
        ur_rest_dict[ur] = ur_cosim
    co_occurence_matrix = pd.DataFrame.from_dict(ur_rest_dict, orient='index')
    co_occurence_matrix.columns=list(restaurants_df.columns)
    return co_occurence_matrix

In [124]:
co_occurence_matrix = cooccurence_matrix_creator()
co_occurence_matrix.head()

,Little_Donkey,Caf__Nero,India_Pavilion,Desi_Dhaba,Thai_Sensation,Dumpling_House,Caf__Luna
Little_Donkey,1.000000,0.606349,0.811778,0.00000,0.862105,0.309839,0.651273
Caf__Nero,0.606349,1.000000,0.170910,0.29173,0.272259,0.782794,0.174034
India_Pavilion,0.811778,0.170910,1.000000,0.00000,0.904497,0.000000,0.706049
Thai_Sensation,0.862105,0.272259,0.904497,0.00000,1.000000,0.000000,0.809807


# Review of some of the Variables

In [125]:
user_rated

['Little_Donkey', 'Caf__Nero', 'India_Pavilion', 'Thai_Sensation']

In [126]:
user_not_rated

['Desi_Dhaba', 'Dumpling_House ', 'Caf__Luna']

In [127]:
co_occurence_matrix

,Little_Donkey,Caf__Nero,India_Pavilion,Desi_Dhaba,Thai_Sensation,Dumpling_House,Caf__Luna
Little_Donkey,1.000000,0.606349,0.811778,0.00000,0.862105,0.309839,0.651273
Caf__Nero,0.606349,1.000000,0.170910,0.29173,0.272259,0.782794,0.174034
India_Pavilion,0.811778,0.170910,1.000000,0.00000,0.904497,0.000000,0.706049
Thai_Sensation,0.862105,0.272259,0.904497,0.00000,1.000000,0.000000,0.809807


In [128]:
def rest_to_rate(df):
    restaurants_to_rate = df[user_not_rated_list(user)]
    return restaurants_to_rate

In [129]:
restaurants_to_rate = rest_to_rate(co_occurence_matrix)
restaurants_to_rate

,Desi_Dhaba,Dumpling_House,Caf__Luna
Little_Donkey,0.00000,0.309839,0.651273
Caf__Nero,0.29173,0.782794,0.174034
India_Pavilion,0.00000,0.000000,0.706049
Thai_Sensation,0.00000,0.000000,0.809807


In [130]:
'''Generating the prediction'''

def rating_predictor_colab_ii():
    predicted_rating_dict = {}
    for not_rated in restaurants_to_rate.columns: 
        column_df = pd.DataFrame(restaurants_to_rate[not_rated].sort_values(ascending=False))
        predicted_rating = []
        top_sim_list = []
        for top_n in range(2):   

            top_sim = column_df[not_rated][top_n]
            top_sim_list.append(top_sim)

            given_rr = column_df[not_rated].index[top_n]
            given_rank_df = pd.DataFrame(restaurants_df.loc[user, :]) # While converting this to function the name should be an argument to the function
            given_rank = given_rank_df.loc[given_rr,:]

            predicted_rating_item = float(given_rank*top_sim)
            predicted_rating.append(predicted_rating_item)
        total_sim_weight = sum(top_sim_list)
        adjusted_predicted_rating = (sum(predicted_rating))/total_sim_weight
        predicted_rating_dict[not_rated] = adjusted_predicted_rating
        
    '''Creating the Dataframe in this step'''
        
    predicted_df = pd.DataFrame.from_dict(predicted_rating_dict, orient='index')
    predicted_df.columns=[user]
    return predicted_df
#     return (predicted_rating_dict)


In [131]:
predicted_df = rating_predictor_colab_ii()
predicted_df.head()

,Tess
Desi_Dhaba,3.000000
Dumpling_House,3.000000
Caf__Luna,3.534224


In [132]:
'''Generating the prediction - All in One Function'''

def rating_predictor_colab_ii_all():
    
    predicted_rating_dict = {}
    
#     user_rated = user_rated_list(user)
#     user_not_rated = user_not_rated_list(user)
#     
    co_occurence_matrix = cooccurence_matrix_creator()
    restaurants_to_rate = rest_to_rate(co_occurence_matrix)
    
    for not_rated in restaurants_to_rate.columns: 
        column_df = pd.DataFrame(restaurants_to_rate[not_rated].sort_values(ascending=False))
        predicted_rating = []
        top_sim_list = []
        for top_n in range(2):   

            top_sim = column_df[not_rated][top_n]
            top_sim_list.append(top_sim)

            given_rr = column_df[not_rated].index[top_n]
            given_rank_df = pd.DataFrame(restaurants_df.loc[user, :]) # While converting this to function the name should be an argument to the function
            given_rank = given_rank_df.loc[given_rr,:]

            predicted_rating_item = float(given_rank*top_sim)
            predicted_rating.append(predicted_rating_item)
        total_sim_weight = sum(top_sim_list)
        adjusted_predicted_rating = (sum(predicted_rating))/total_sim_weight
        predicted_rating_dict[not_rated] = adjusted_predicted_rating
        
    '''Creating the Dataframe in this step'''
        
    predicted_df = pd.DataFrame.from_dict(predicted_rating_dict, orient='index')
    predicted_df.columns=[user]
    return predicted_df
#     return (predicted_rating_dict)

In [133]:
'''Testing the function to deal with users who have rated only one restaurant'''

'''Collaborative Item-Item filtering function to make predictions'''

def rating_predictor_colab_ii_all():
    
    predicted_rating_dict = {}
    
#     user_rated = user_rated_list(user)
#     user_not_rated = user_not_rated_list(user)
     
    co_occurence_matrix = cooccurence_matrix_creator()
    restaurants_to_rate = rest_to_rate(co_occurence_matrix)
    
    for not_rated in restaurants_to_rate.columns: 
        column_df = pd.DataFrame(restaurants_to_rate[not_rated].sort_values(ascending=False))
        predicted_rating = []
        top_sim_list = []
        if column_df.shape[0] == 1: 
            top_sim = column_df[not_rated]
            
#             print ('Top Sim: ', top_sim)
            top_sim_list.append(top_sim)

            given_rr = column_df[not_rated].index
#             print ('Given RR:', given_rr)
            
            given_rank_df = pd.DataFrame(restaurants_df.loc[user, :]) # While converting this to function the name should be an argument to the function
            given_rank = float((given_rank_df.loc[given_rr,:]).get_values())
#             print ('Given Rank:',type(given_rank))

            adjusted_predicted_rating = float(given_rank*top_sim)
            predicted_rating_dict[not_rated] = adjusted_predicted_rating
             
        else: 
            for top_n in range(co_occurence_matrix.shape[0]):   

                top_sim = column_df[not_rated][top_n]
#                 print ('Top Sim: ', top_sim)
                top_sim_list.append(top_sim)

                given_rr = column_df[not_rated].index[top_n]
#                 print ('Given RR:', given_rr)
                
                given_rank_df = pd.DataFrame(restaurants_df.loc[user, :]) # While converting this to function the name should be an argument to the function
                given_rank = given_rank_df.loc[given_rr,:]
#                 print ('Given Rank:',type(given_rank))

                predicted_rating_item = float(given_rank*top_sim)
                predicted_rating.append(predicted_rating_item)
            total_sim_weight = sum(top_sim_list)
            adjusted_predicted_rating = (sum(predicted_rating))/total_sim_weight
            predicted_rating_dict[not_rated] = adjusted_predicted_rating
        
    '''Creating the Dataframe in this step'''
        
    predicted_df = pd.DataFrame.from_dict(predicted_rating_dict, orient='index')
    predicted_df.fillna(0, inplace=True)
    predicted_df.columns=[user]
    predicted_df.sort_values([user], ascending=False, inplace=True)
    return predicted_df


In [134]:
predicted_df = rating_predictor_colab_ii_all()
predicted_df.head()

,Tess
Caf__Luna,3.345899
Desi_Dhaba,3.000000
Dumpling_House,3.000000


# Collaborative User to User Filtering

In [135]:
restaurants_df

,Little_Donkey,Caf__Nero,India_Pavilion,Desi_Dhaba,Thai_Sensation,Dumpling_House,Caf__Luna
Users,,,,,,,
Ajinth,4.0,0.0,5.0,0.0,3.0,0.0,2.0
Tess,3.0,3.0,3.0,0.0,4.0,0.0,0.0
Jessica,4.0,3.0,0.0,0.0,2.0,0.0,1.0
Mark,0.0,2.0,0.0,2.0,0.0,2.0,4.0
Joe,5.0,0.0,5.0,0.0,8.0,0.0,8.0
Chris,3.0,5.0,0.0,0.0,0.0,4.0,0.0


In [136]:
# user_rated

In [137]:
# user_not_rated

In [138]:
'''This function calcultes the co-sine similarity between different users for the user-user collaborative filtering model'''

def user_cooccurence_matrix_creator():
    user_dict = {}
    user_columns = []
    user_cosim=[]
    
    for ur in restaurants_df.iterrows(): 
        if ur[0] == user: 
            continue 
        else: 
            user_similarity = cosim(restaurants_df.loc[ur[0],:].values, restaurants_df.loc[user,:].values)
            user_cosim.append(user_similarity)
            user_columns.append(ur[0])
    user_dict[user] = user_cosim
    
    user_df = pd.DataFrame.from_dict(user_dict, orient='index')
    user_df.columns = user_columns
    return user_df
    


In [139]:
ur_coocurence_df = user_cooccurence_matrix_creator()
ur_coocurence_df

,Ajinth,Jessica,Mark,Joe,Chris
Tess,0.809345,0.807427,0.172917,0.708676,0.517597


In [140]:
'''Sorting the dataframe here in descending order'''
ur_coocurence_df.sort_index(ascending=False, axis=0, inplace=True)
ur_coocurence_df

,Ajinth,Jessica,Mark,Joe,Chris
Tess,0.809345,0.807427,0.172917,0.708676,0.517597


In [141]:
ur_coocurence_df_t= ur_coocurence_df.T
ur_coocurence_df_t.columns=['Similarity']
ur_coocurence_df_t

,Similarity
Ajinth,0.809345
Jessica,0.807427
Mark,0.172917
Joe,0.708676
Chris,0.517597


In [142]:
ur_restaurants_to_rate = rest_to_rate(restaurants_df)
ur_restaurants_to_rate

,Desi_Dhaba,Dumpling_House,Caf__Luna
Users,,,
Ajinth,0.0,0.0,2.0
Tess,0.0,0.0,0.0
Jessica,0.0,0.0,1.0
Mark,2.0,2.0,4.0
Joe,0.0,0.0,8.0
Chris,0.0,4.0,0.0


In [143]:
ur_restaurants_to_rate.drop(user, axis = 0, inplace=True )
ur_restaurants_to_rate

/Users/ajinthchristudas/anaconda/lib/python3.6/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


,Desi_Dhaba,Dumpling_House,Caf__Luna
Users,,,
Ajinth,0.0,0.0,2.0
Jessica,0.0,0.0,1.0
Mark,2.0,2.0,4.0
Joe,0.0,0.0,8.0
Chris,0.0,4.0,0.0


In [144]:
joint_matrix = ur_restaurants_to_rate.join(ur_coocurence_df_t, how='inner')
# joint_df = joint_matrix.iloc[:2,:]
joint_df = joint_matrix.iloc[:3,:]
joint_df

,Desi_Dhaba,Dumpling_House,Caf__Luna,Similarity
Users,,,,
Ajinth,0.0,0.0,2.0,0.809345
Jessica,0.0,0.0,1.0,0.807427
Mark,2.0,2.0,4.0,0.172917


In [145]:
'''Working on some more updates to the cell above'''

rating_predictor_df = defaultdict(list)
for k,v in enumerate(joint_df.iterrows()): 
    for i in range(len(v[1].index)):
        rating = (v[1][i]) 
        restaurant = (v[1].index[i])  
        rating_predictor_df[restaurant].append(rating)
        
pprint.pprint (rating_predictor_df)

defaultdict(<class 'list'>,
            {'Caf__Luna': [2.0, 1.0, 4.0],
             'Desi_Dhaba': [0.0, 0.0, 2.0],
             'Dumpling_House ': [0.0, 0.0, 2.0],
             'Similarity': [0.80934464827409736,
                            0.80742676726507157,
                            0.17291712531127049]})


In [146]:
'''Making Predictions for the User - User Collaborative Filtering'''
rating_dict = defaultdict(int)

for k,v in rating_predictor_df.items():
    new_rating = sum(np.array(v)*np.array(rating_predictor_df['Similarity']))/sum(np.array(rating_predictor_df['Similarity']))
    rating_dict[k] = new_rating

pprint.pprint (rating_dict)

rating_df = pd.DataFrame.from_dict(rating_dict, orient='index')
rating_df.columns=[user]
rating_df.sort_values(user, ascending=False, inplace=True)
rating_df.head()  

defaultdict(<class 'int'>,
            {'Caf__Luna': 1.7420822081013119,
             'Desi_Dhaba': 0.19323711513412525,
             'Dumpling_House ': 0.19323711513412525,
             'Similarity': 0.74698867758112342})


,Tess
Caf__Luna,1.742082
Similarity,0.746989
Desi_Dhaba,0.193237
Dumpling_House,0.193237
